In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.ticker import ScalarFormatter, NullFormatter

plt.rcParams.update({'font.size': 14})
matplotlib.rcParams['xtick.minor.size'] = 0
matplotlib.rcParams['xtick.minor.width'] = 0

import sys
sys.path.append("..")

from ContrastEstimation import util, IntensityEstimation

# Define the sample molecule